In [16]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

In [17]:
stop_times = pd.read_csv('dados\paradas\stop_times.txt', sep=',')
stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'])
stop_times['hour'] = stop_times['arrival_time'].dt.hour
stop_times['minute'] = stop_times['arrival_time'].dt.minute
print(stop_times[['arrival_time', 'hour', 'minute']].head(10))

         arrival_time  hour  minute
0 2023-02-27 18:00:00    18       0
1 2023-02-27 18:02:00    18       2
2 2023-02-27 18:04:00    18       4
3 2023-02-27 18:06:00    18       6
4 2023-02-27 18:08:00    18       8
5 2023-02-27 18:10:00    18      10
6 2023-02-27 18:12:00    18      12
7 2023-02-27 18:14:00    18      14
8 2023-02-27 18:16:00    18      16
9 2023-02-27 18:18:00    18      18


In [18]:
stops = pd.read_csv('dados\paradas\stops.txt', sep=',')
print(stops.describe)

<bound method NDFrame.describe of          stop_id                              stop_name  \
0          18848                               Clínicas   
1          18849                          Vila Madalena   
2          18850                             Consolação   
3          18851                              Conceição   
4          18852                              Jabaquara   
...          ...                                    ...   
20923  960011478                  Av. Pirajussara, 4370   
20924  960011507             R. Caminho Do Engenho, 255   
20925  960011508             R. Caminho Do Engenho, 417   
20926  960011509  Av. Dr. Guilherme Dumont Villares, 50   
20927  960011510        R. Alfredo Mendes Da Silva, 499   

                                               stop_desc   stop_lat   stop_lon  
0                                                    NaN -23.554022 -46.671108  
1                                                    NaN -23.546498 -46.691141  
2             

In [19]:
events = pd.merge(stop_times, stops, on='stop_id')
print(events['arrival_time'].head())

0   2023-02-27 18:00:00
1   2023-02-27 12:37:00
2   2023-02-27 18:02:00
3   2023-02-27 18:04:00
4   2023-02-27 17:59:44
Name: arrival_time, dtype: datetime64[ns]


In [60]:
filtro = events.loc[(events['stop_id']==440016922)].sort_values(by='arrival_time')

filtro['freq'] = filtro['arrival_time'] - filtro['arrival_time'].shift()

print(filtro['freq'].median())

0 days 00:15:25.500000


In [ ]:
filtro = events.groupby("stop_id")

In [20]:
filter_morning = events.loc[(events['hour'] >= 17) & (events['hour'] < 19)].groupby('stop_id', as_index=False).count()
filter_morning = filter_morning[['stop_id', 'trip_id']]
filter_morning = filter_morning.rename(columns={'trip_id': 'qtd'})

filter_morning = pd.merge(stops,filter_morning, on='stop_id')

print(filter_morning.loc[(filter_morning['qtd'] >= 10)].count())
print(filter_morning.count())


stop_id      731
stop_name    731
stop_desc    720
stop_lat     731
stop_lon     731
qtd          731
dtype: int64
stop_id      15418
stop_name    15418
stop_desc    14595
stop_lat     15418
stop_lon     15418
qtd          15418
dtype: int64


In [21]:
fig = go.Figure()

fig = px.scatter_mapbox(filter_morning, lat="stop_lat", lon="stop_lon", zoom=9, width=600, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

def SetColor(df):
    values = df['qtd'].tolist()
    color_list = []
    for i in values:
        if(i >= 10):
            color_list.append("red")
        else:
            color_list.append("blue")
    
    return color_list

def SetSize(df):
    values = df['qtd'].tolist()
    color_list = []
    for i in values:
        if(i >= 10):
            color_list.append(10)
        else:
            color_list.append(4)
    
    return color_list

fig.add_trace(go.Scattermapbox(
        lat=filter_morning['stop_lat'].tolist(),
        lon=filter_morning['stop_lon'].tolist(),
        mode='markers',
        marker=go.scattermapbox.Marker(
            size= SetSize(filter_morning),
            color= SetColor(filter_morning)  #function gets called here and will return a list of colors, (i.e. ['green', 'blue', 'red', 'green'])
        ),
    )
)

fig.show()